In [1]:
!pip install astroNN --quiet
!pip install torchmetrics --quiet
!pip install scikit-image --quiet
!pip install mlflow --quiet

In [28]:
import torch
from torch import nn
from torch.utils.data import DataLoader,Dataset
import torch.nn.functional as F
import torchmetrics

from collections import Counter

from utils.focal_loss import FocalLoss
import utils.general as g
from cnn import NeuralNet

import os
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

import numpy as np
import matplotlib.pyplot as plt

from astroNN.datasets import load_galaxy10

import skimage as ski

torch.manual_seed(42)
np.random.seed(42)
torch.__version__

'2.7.1+cu126'

In [29]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [30]:
class_names = [
    "0 - Disturbed Galaxies",
    "1 - Merging Galaxies",
    "2 - Round Smooth Galaxies",
    "3 - In-between Round Smooth Galaxies",
    "4 - Cigar Shaped Smooth Galaxies",
    "5 - Barred Spiral Galaxies",
    "6 - Unbarred Tight Spiral Galaxies",
    "7 - Unbarred Loose Spiral Galaxies",
    "8 - Edge-on Galaxies without Bulge",
    "9 - Edge-on Galaxies with Bulge"
]

In [31]:
!pwd

/home/arpoca/GalaxyClassifier/src


In [32]:
images,labels = g.get_data('/home/arpoca/GalaxyClassifier/src/data')

In [33]:
def get_label_count(labels):
  counts = np.unique(labels, return_counts=True)
  return counts[1].tolist()

In [34]:
X_train, X_temp, y_train, y_temp = train_test_split(
    images, labels, test_size=0.4, stratify=labels, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)
X_temp = None
y_temp = None

# Verify splits
print(f"Train distribution: {Counter(y_train)}")
print(f"Valid distribution: {Counter(y_val)}")
print(f"Test distribution: {Counter(y_test)}")

Train distribution: Counter({np.uint8(2): 1587, np.uint8(7): 1577, np.uint8(5): 1226, np.uint8(3): 1216, np.uint8(9): 1124, np.uint8(1): 1112, np.uint8(6): 1097, np.uint8(8): 854, np.uint8(0): 648, np.uint8(4): 200})
Valid distribution: Counter({np.uint8(2): 529, np.uint8(7): 525, np.uint8(5): 408, np.uint8(3): 405, np.uint8(9): 374, np.uint8(1): 371, np.uint8(6): 366, np.uint8(8): 285, np.uint8(0): 217, np.uint8(4): 67})
Test distribution: Counter({np.uint8(2): 529, np.uint8(7): 526, np.uint8(5): 409, np.uint8(3): 406, np.uint8(9): 375, np.uint8(1): 370, np.uint8(6): 366, np.uint8(8): 284, np.uint8(0): 216, np.uint8(4): 67})


In [35]:
num_samples_per_class = get_label_count(y_train)
classes = np.unique(y_train)
class_weights = compute_class_weight('balanced', classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))

In [36]:
images_path = '/home/arpoca/GalaxyClassifier/src/data/images.npy'
labels_path = '/home/arpoca/GalaxyClassifier/src/data/labels.npy'
train_dataset, test_dataset, valid_dataset = g.get_dataset(X_train, X_test, X_val,images_path,labels_path)

In [37]:
X_train = y_train = X_test = y_test = X_val = y_val = images = labels = None

In [59]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

In [60]:
model = NeuralNet().to(device)
fl = FocalLoss(
    beta=0.999,
    gamma=2.0,
    samples_per_class=num_samples_per_class,
    reduce=True
).to(device)
criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor(list(class_weight_dict.values()))).to(device)

In [61]:
def train_book(model, optimizer, criterion, train_loader, n_epochs, device):
    accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=10).to(device)
    for epoch in range(n_epochs):
        model.train()
        total_loss = 0.0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            y_pred = model(X_batch).to(device)

            # Ensure y_batch is integer indices
            if y_batch.dtype != torch.long:
                y_batch = y_batch.long()

            loss = criterion(y_pred, y_batch)
            total_loss += loss.item()
            loss.backward()
            optimizer.step()

        model.eval()
        accuracy.reset()
        with torch.no_grad():
            for X_batch, y_batch in valid_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                y_pred = model(X_batch)
                _, predicted_class = torch.max(y_pred, 1)  # Get predicted class indices
                accuracy.update(predicted_class, y_batch)  # Use y_batch directly (integer indices)

        final_accuracy = accuracy.compute()
        with torch.no_grad():
            all_predictions = []
            for X_batch, y_batch in valid_loader:
                X_batch = X_batch.to(device)
                y_pred = model(X_batch)
                _, predicted_class = torch.max(y_pred, 1)
                all_predictions.extend(predicted_class.cpu().numpy())
        pred_distribution = Counter(all_predictions)
        print(f"Predicted class distribution: {pred_distribution}")
        mean_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch + 1}/{n_epochs}, Loss: {mean_loss:.4f}, Validation Accuracy: {final_accuracy:.4f}")

In [62]:
optimizer = torch.optim.SGD(model.parameters(), lr=.15)
train_book(model, optimizer, fl, train_loader, n_epochs=10,device=device)
torch.cuda.empty_cache()

Predicted class distribution: Counter({np.int64(2): 1521, np.int64(5): 1490, np.int64(3): 474, np.int64(4): 48, np.int64(1): 8, np.int64(0): 6})
Epoch 1/10, Loss: 1.1575, Validation Accuracy: 0.1142
Predicted class distribution: Counter({np.int64(5): 1022, np.int64(3): 965, np.int64(1): 859, np.int64(2): 576, np.int64(0): 125})
Epoch 2/10, Loss: 0.9756, Validation Accuracy: 0.2856
Predicted class distribution: Counter({np.int64(2): 1568, np.int64(1): 836, np.int64(0): 595, np.int64(5): 275, np.int64(3): 141, np.int64(4): 74, np.int64(6): 58})
Epoch 3/10, Loss: 0.8927, Validation Accuracy: 0.4590
Predicted class distribution: Counter({np.int64(5): 1615, np.int64(1): 1266, np.int64(4): 231, np.int64(2): 203, np.int64(3): 130, np.int64(0): 90, np.int64(6): 12})
Epoch 4/10, Loss: 0.8365, Validation Accuracy: 0.3039
Predicted class distribution: Counter({np.int64(0): 904, np.int64(1): 732, np.int64(5): 652, np.int64(3): 467, np.int64(2): 399, np.int64(4): 368, np.int64(6): 25})
Epoch 5/10, 

In [51]:
model.eval()
metric = torchmetrics.classification.MulticlassConfusionMatrix(num_classes=10).to(device)

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        y_pred_logits = model(X_batch)  # Raw logits [batch_size, 10]

        # Convert logits to predicted class indices
        _, y_pred_classes = torch.max(y_pred_logits, 1)

        metric.update(y_pred_classes, y_batch)  # Now using class indices, not logits

conf_matrix = metric.compute()

# Calculate precision, recall, and F1-score from the confusion matrix
for i, class_name in enumerate(class_names):
    tp = conf_matrix[i, i].item()
    fp = conf_matrix[:, i].sum().item() - tp
    fn = conf_matrix[i, :].sum().item() - tp

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    print(f"Metrics for {class_name}:")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1-score: {f1_score:.4f}")

Metrics for 0 - Disturbed Galaxies:
  Precision: 0.0000
  Recall: 0.0000
  F1-score: 0.0000
Metrics for 1 - Merging Galaxies:
  Precision: 0.5206
  Recall: 0.9892
  F1-score: 0.6822
Metrics for 2 - Round Smooth Galaxies:
  Precision: 0.0000
  Recall: 0.0000
  F1-score: 0.0000
Metrics for 3 - In-between Round Smooth Galaxies:
  Precision: 0.0000
  Recall: 0.0000
  F1-score: 0.0000
Metrics for 4 - Cigar Shaped Smooth Galaxies:
  Precision: 0.0000
  Recall: 0.0000
  F1-score: 0.0000
Metrics for 5 - Barred Spiral Galaxies:
  Precision: 0.0000
  Recall: 0.0000
  F1-score: 0.0000
Metrics for 6 - Unbarred Tight Spiral Galaxies:
  Precision: 0.0000
  Recall: 0.0000
  F1-score: 0.0000
Metrics for 7 - Unbarred Loose Spiral Galaxies:
  Precision: 0.0000
  Recall: 0.0000
  F1-score: 0.0000
Metrics for 8 - Edge-on Galaxies without Bulge:
  Precision: 0.0000
  Recall: 0.0000
  F1-score: 0.0000
Metrics for 9 - Edge-on Galaxies with Bulge:
  Precision: 0.0000
  Recall: 0.0000
  F1-score: 0.0000


In [52]:
print("=== DEBUGGING EVALUATION ===")
model.eval()

# Check what the model actually predicts during evaluation
all_preds = []
all_targets = []

with torch.no_grad():
    for X_batch, y_batch in valid_loader:  # Use the SAME loader as training
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        y_pred_logits = model(X_batch)
        _, y_pred_classes = torch.max(y_pred_logits, 1)

        all_preds.extend(y_pred_classes.cpu().numpy())
        all_targets.extend(y_batch.cpu().numpy())

from collections import Counter
print(f"EVALUATION - Predicted distribution: {Counter(all_preds)}")
print(f"EVALUATION - Target distribution: {Counter(all_targets)}")

# Now run your confusion matrix code
metric = torchmetrics.classification.MulticlassConfusionMatrix(num_classes=10).to(device)

with torch.no_grad():
    for X_batch, y_batch in valid_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        y_pred_logits = model(X_batch)
        _, y_pred_classes = torch.max(y_pred_logits, 1)
        metric.update(y_pred_classes, y_batch)

conf_matrix = metric.compute()
print(f"Confusion matrix shape: {conf_matrix.shape}")
print(f"Confusion matrix:\n{conf_matrix}")


=== DEBUGGING EVALUATION ===
EVALUATION - Predicted distribution: Counter({np.int64(1): 3520, np.int64(5): 27})
EVALUATION - Target distribution: Counter({np.int64(1): 1853, np.int64(0): 1081, np.int64(2): 613})
Confusion matrix shape: torch.Size([10, 10])
Confusion matrix:
tensor([[   0, 1076,    0,    0,    0,    5,    0,    0,    0,    0],
        [   0, 1833,    0,    0,    0,   20,    0,    0,    0,    0],
        [   0,  611,    0,    0,    0,    2,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0]],
       device='cuda:0')


In [58]:
len(valid_loader)    

56